# Recognizing Human Activities using Smartphones
## Classification on Smartphone-Based Recognition of Human Activities and Postural Transitions Dataset

In [17]:
# Data source: http://archive.ics.uci.edu/ml/datasets/Smartphone-Based+Recognition+of+Human+Activities+and+Postural+Transitions#
url <- "http://archive.ics.uci.edu/ml/machine-learning-databases/00341/HAPT%20Data%20Set.zip"

# Specifying destination where file should be saved
destfile <- "data/localfile.zip"

# Apply download.file function 
download.file(url, destfile)

unzip ("data/localfile.zip", exdir = "data/")

In [48]:
# create a vector of features to use for col names
features_vector <- scan("data/features.txt", character(), quote = "")

In [53]:
# create a vector of activity labels to add such a variable as a column into the data frame for each observation. 
y_training_activity_label <- scan("data/Train/y_train.txt", character(), quote = "")

In [56]:
# read feature values 
x_training_data <- read_delim("data/Train/X_train.txt", delim = " ", col_names = features_vector)

Warning message:
“Duplicated column names deduplicated: 'tBodyAcc-ropy-1' => 'tBodyAcc-ropy-1_1' [24], 'tBodyAcc-ropy-1' => 'tBodyAcc-ropy-1_2' [25], 'tGravityAcc-ropy-1' => 'tGravityAcc-ropy-1_1' [64], 'tGravityAcc-ropy-1' => 'tGravityAcc-ropy-1_2' [65], 'tBodyAccJerk-ropy-1' => 'tBodyAccJerk-ropy-1_1' [104], 'tBodyAccJerk-ropy-1' => 'tBodyAccJerk-ropy-1_2' [105], 'tBodyGyro-ropy-1' => 'tBodyGyro-ropy-1_1' [144], 'tBodyGyro-ropy-1' => 'tBodyGyro-ropy-1_2' [145], 'tBodyGyroJerk-ropy-1' => 'tBodyGyroJerk-ropy-1_1' [184], 'tBodyGyroJerk-ropy-1' => 'tBodyGyroJerk-ropy-1_2' [185], 'fBodyAcc-ropy-1' => 'fBodyAcc-ropy-1_1' [289], 'fBodyAcc-ropy-1' => 'fBodyAcc-ropy-1_2' [290], 'fBodyAcc-Skewness-1' => 'fBodyAcc-Skewness-1_1' [299], 'fBodyAcc-Kurtosis-1' => 'fBodyAcc-Kurtosis-1_1' [300], 'fBodyAcc-Skewness-1' => 'fBodyAcc-Skewness-1_2' [301], 'fBodyAcc-Kurtosis-1' => 'fBodyAcc-Kurtosis-1_2' [302], 'fBodyAccJerk-ropy-1' => 'fBodyAccJerk-ropy-1_1' [368], 'fBodyAccJerk-ropy-1' => 'fBodyAccJerk-r

In [58]:
# add activity label column
x_training_data <- x_training_data %>%
    mutate (activity_label = y_training_activity_label)

x_training_data

tBodyAcc-Mean-1,tBodyAcc-Mean-2,tBodyAcc-Mean-3,tBodyAcc-STD-1,tBodyAcc-STD-2,tBodyAcc-STD-3,tBodyAcc-Mad-1,tBodyAcc-Mad-2,tBodyAcc-Mad-3,tBodyAcc-Max-1,⋯,fBodyGyroJerkMag-Skewness-1,fBodyGyroJerkMag-Kurtosis-1,tBodyAcc-AngleWRTGravity-1,tBodyAccJerk-AngleWRTGravity-1,tBodyGyro-AngleWRTGravity-1,tBodyGyroJerk-AngleWRTGravity-1,tXAxisAcc-AngleWRTGravity-1,tYAxisAcc-AngleWRTGravity-1,tZAxisAcc-AngleWRTGravity-1,activity_label
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
0.043579674,-0.0059702213,-0.035054344,-0.9953812,-0.98836586,-0.9373820,-0.9950070,-0.98881558,-0.9533252,-0.79479637,⋯,-0.31484835,-0.713307812,-0.112754341,0.030400372,-0.464761386,-0.018445884,-0.8415585,0.17991281,-0.05171842,5
0.039480037,-0.0021312757,-0.029067362,-0.9983480,-0.98294494,-0.9712729,-0.9987020,-0.98331483,-0.9739998,-0.80253665,⋯,-0.60319922,-0.860676901,0.053476955,-0.007434566,-0.732626210,0.703510588,-0.8450924,0.18026111,-0.04743634,5
0.039977781,-0.0051527161,-0.022650708,-0.9954821,-0.97731384,-0.9847595,-0.9964148,-0.97583480,-0.9859730,-0.79847717,⋯,-0.40442749,-0.761847228,-0.118559255,0.177899475,0.100699208,0.808529075,-0.8492301,0.18060956,-0.04227136,5
0.039784558,-0.0118087780,-0.028915779,-0.9961941,-0.98856859,-0.9932556,-0.9969943,-0.98852643,-0.9931354,-0.79847717,⋯,-0.13837282,-0.491604347,-0.036787973,-0.012892494,0.640011043,-0.485366445,-0.8489466,0.18190709,-0.04082622,5
0.038758138,-0.0022885329,-0.023862885,-0.9982413,-0.98677414,-0.9931155,-0.9982159,-0.98647918,-0.9938246,-0.80198153,⋯,-0.36621362,-0.702489762,0.123320048,0.122541962,0.693578288,-0.615970609,-0.8481640,0.18512369,-0.03707990,5
0.038988008,0.0041088524,-0.017340265,-0.9974376,-0.99348542,-0.9966920,-0.9975222,-0.99349350,-0.9969160,-0.80198153,⋯,-0.55490231,-0.844224053,0.082632155,-0.143439015,0.275040750,-0.368224043,-0.8499269,0.18479502,-0.03532556,5
0.039897454,-0.0053243398,-0.020456504,-0.9970237,-0.97731310,-0.9877824,-0.9968975,-0.97744996,-0.9893912,-0.80060627,⋯,-0.23557601,-0.571125698,-0.212754062,-0.230621927,0.014636690,-0.189511526,-0.8524406,0.18214232,-0.03620339,5
0.039082257,-0.0160471309,-0.030241314,-0.9966619,-0.97699556,-0.9866725,-0.9963801,-0.97759391,-0.9893098,-0.80060627,⋯,-0.10433669,-0.432022209,-0.020888015,0.593995810,-0.561870666,0.467383325,-0.8513093,0.18375096,-0.03517599,5
0.039026181,-0.0074099999,-0.027300691,-0.9974311,-0.97318992,-0.9881834,-0.9974906,-0.97155660,-0.9901557,-0.80024535,⋯,-0.28636581,-0.579473824,0.012954021,0.080936389,-0.234312625,0.117797005,-0.8482700,0.18895526,-0.03059418,5
